In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os

# 1. Check for GPU
print("TensorFlow Version:", tf.__version__)
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"✅ GPU Detected: {gpus}")
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)
else:
    print("⚠️ No GPU detected. Training will run on CPU (slower).")

In [ ]:
# 2. Load Data
# Make sure the CSV file is in the same directory as this notebook
csv_path = 'SQLInjection_XSS_MixDataset.1.0.0.csv'

if os.path.exists(csv_path):
    df = pd.read_csv(csv_path)
    print(f"✅ Loaded dataset with {len(df)} rows")
else:
    print(f"❌ File not found: {csv_path}")
    print("Please download the dataset and place it in this folder.")

In [ ]:
import re
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

# Preprocessing Functions
def remove_comment(text):
    text = str(text)
    text = re.sub(r'//.*?
|/\*.*?\*/', '', text, flags=re.S)
    text = text.split('--')[0]+"--"
    if '\'' in text:
        removeTarget = text.split('\'')[0]
        text = text.replace(removeTarget, "")
    return text

def data2char_index(X, max_len, is_remove_comment=False):
    alphabet = " abcdefghijklmnopqrstuvwxyz0123456789-,;.!?:'\"/\\|_@#$%^&*~`+-=<>()[]{}"
    result = [] 
    for data in X:
        mat = []
        if is_remove_comment:
            data = remove_comment(data)
        for ch in str(data):
            ch = ch.lower()
            if ch not in alphabet:
                continue
            mat.append(alphabet.index(ch))
        result.append(mat)
    return pad_sequences(np.array(result, dtype=object), padding='post', truncating='post', maxlen=max_len)

def data_to_symbol_tag(X, max_len, is_remove_comment=False):
    symbol = " -,;.!?:'\"/\\|_@#$%^&*~`+-=<>()[]{}"
    result = [] 
    for data in X:
        mat = []
        if is_remove_comment:
            data = remove_comment(data)
        for ch in str(data):
            ch = ch.lower()
            if ch not in symbol:
                mat.append(0)
            else:
                mat.append(symbol.index(ch))
        result.append(mat)
    return pad_sequences(np.array(result, dtype=object), padding='post', truncating='post', maxlen=max_len)

In [ ]:
# Prepare Data
if 'df' in locals():
    data = df['Sentence'].values
    # Create labels
    SQLInjection_label = df['SQLInjection'].values
    XSS = df['XSS'].values
    Normal = df['Normal'].values
    label = np.array([SQLInjection_label, XSS, Normal]).T

    # Split Data
    trainX, testX, y_train, y_test = train_test_split(data, label, test_size=0.2, random_state=42)
    trainX, x_val, y_train, y_val = train_test_split(trainX, y_train, test_size=0.2, random_state=42)

    print("Processing text data... (this may take a moment)")
    MAX_LEN = 1000
    
    trainX_text = data2char_index(trainX, max_len=MAX_LEN)
    trainX_symbol = data_to_symbol_tag(trainX, max_len=MAX_LEN)
    
    x_val_text = data2char_index(x_val, max_len=MAX_LEN)
    x_val_symbol = data_to_symbol_tag(x_val, max_len=MAX_LEN)
    
    testX_text = data2char_index(testX, max_len=MAX_LEN)
    testX_symbol = data_to_symbol_tag(testX, max_len=MAX_LEN)
    
    print("Data processing complete.")
    print(f"Train shape: {trainX_text.shape}")

In [ ]:
# Define Model (Updated for Keras 3)
def create_model(max_len):
    pool_siz = 10
    num_heads = 3
    
    # Text Input
    input_text = tf.keras.layers.Input(shape=(max_len,), name="text_input")
    embed1 = tf.keras.layers.Embedding(input_dim=70, output_dim=105, input_length=max_len, trainable=False)(input_text)
    cnn1 = tf.keras.layers.Conv1D(32, 3, padding='same', strides=1, activation='relu')(embed1)
    cnn1 = tf.keras.layers.MaxPooling1D(pool_size=pool_siz)(cnn1)
    
    # GRU 1 (Bidirectional)
    # In Keras 3, we can use go_backwards=True safely during training
    GRU0 = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32, return_sequences=True, go_backwards=True))(cnn1)
    
    # Symbol Input
    input_symbol = tf.keras.layers.Input(shape=(max_len,), name="symbol_input")
    embed2 = tf.keras.layers.Embedding(input_dim=34, output_dim=51, input_length=max_len, trainable=False)(input_symbol)
    cnn1s = tf.keras.layers.Conv1D(32, 3, padding='same', strides=1, activation='relu')(embed2)
    cnn1s = tf.keras.layers.MaxPooling1D(pool_size=pool_siz)(cnn1s)
    
    # GRU 2 (Bidirectional)
    GRU0s = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32, return_sequences=True, go_backwards=True))(cnn1s)
    
    # Cross Attention
    CrossAT1 = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=1)(GRU0, GRU0s)
    
    # Combine
    combined = tf.keras.layers.add([GRU0 + CrossAT1, GRU0s + CrossAT1])
    flat = tf.keras.layers.Flatten()(combined)
    dnn1 = tf.keras.layers.Dense(3, activation="softmax")(flat)
    
    model = tf.keras.Model(inputs=[input_text, input_symbol], outputs=dnn1)
    return model

model = create_model(1000)
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.summary()

In [ ]:
# Train Model
if 'trainX_text' in locals():
    history = model.fit(
        [trainX_text, trainX_symbol], 
        y_train, 
        batch_size=64, 
        epochs=10, 
        validation_data=([x_val_text, x_val_symbol], y_val)
    )

In [ ]:
# Save Model in NEW Format (.keras)
model.save('sqli_xss_model_latest.keras')
print("✅ Model saved as 'sqli_xss_model_latest.keras'")

# Also save weights just in case
model.save_weights('sqli_xss_weights_latest.weights.h5')
print("✅ Weights saved as 'sqli_xss_weights_latest.weights.h5'")